In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

nltk.download('words')
words = set(nltk.corpus.words.words())


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/wcrossb/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package words to /Users/wcrossb/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
df1 = pd.read_csv("data/daily_db.csv")
df2 = pd.read_csv("data/stored_vars_hr_update.csv")

df = pd.concat([df1, df2])
print(df.shape)
df = df.drop_duplicates(subset='tweet', keep="first")
print(df.shape)


print(df.shape)
print(df1.tail())



/var/folders/3_/4hp02k9s28l0xngsnvfp3jl40000gn/T/ipykernel_21267/2733460770.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("data/daily_db.csv")


(1117390, 6)
(1114422, 6)
(1114422, 6)
        Unnamed: 0.2  Unnamed: 0                 date  \
1058320       409592       111.0  2021-02-28 23:03:04   
1058321       409593       112.0  2021-02-28 23:03:00   
1058322       409594       113.0  2021-02-28 23:02:51   
1058323       409595       114.0  2021-02-28 23:01:03   
1058324       409597       116.0  2021-02-28 23:00:05   

                                                     tweet  Unnamed: 0.1  
1058320  Senate To Begin Deliberations This Week On Lat...      409592.0  
1058321  Rep. @RashidaTlaib questions Fed Chair Powell:...      409593.0  
1058322  It’s a new month. I pray it’s a green month on...      409594.0  
1058323  #StockUpdate $GOOGL one of my favorite setups....      409595.0  
1058324  40% stock market crash is not coming but 'all ...      409597.0  


In [3]:
#df = pd.read_csv("stored_vars.csv", lineterminator='\n')
pd.read_csv("data/daily_db.csv")

def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",str(tweet)) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", str(tweet)) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet)
         if w.lower() in words or not w.isalpha())
    return tweet

df['tweet_clean'] = df['tweet'].apply(cleaner)


/var/folders/3_/4hp02k9s28l0xngsnvfp3jl40000gn/T/ipykernel_21267/2487201749.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("data/daily_db.csv")


In [4]:
list1 = []
for i in df['tweet_clean']:
    list1.append((sid.polarity_scores(str(i)))['compound'])

df['sentiment'] = pd.Series(list1)
#df.to_csv("inspect.csv")


In [6]:
#Import csv of keywords derived from michigan survey
df_kws = pd.read_csv("kws/kws.csv")
kw_list = df_kws['Indicator'].values.tolist()
category_list = df_kws['Category'].values.tolist()
category_list = [*set(category_list)]

#Detect which keyword was used for search, and store line by line
df['Indicator'] = np.nan
for term in kw_list:
    print(term)
    df['Indicator'] = np.where(df['tweet_clean'].str.contains(term, case=False) == True, term, df['Indicator'])

#Drop Duplicates and compare how many 
print("Length before drop" , df.shape)
df = df.drop_duplicates(subset='tweet_clean', keep="first")
print("length after drop duplicates" , df.shape)
df = pd.merge(df_kws, df, on='Indicator')
#df.to_csv("inspect2.csv")

Better financially
Worse financially
Business conditions
Business environment
State of the economy
Unemployment
Economic Depression
Government Economic Policy
Out of work
Economic policy
Interest rates
Rising prices
Disposable income
Incomes
Family income
Gasoline prices
Own a home
Market value of home
House prices
Inflation
Standard of living
Stock market
Cost of living
Length before drop (1114422, 9)
length after drop duplicates (1059355, 9)


In [7]:
#df= pd.read_csv("inspect2.csv", lineterminator='\n')

#df['sentiment_bool'] = np.where(df['sentiment'] > 0, 1, -1)
#df_p = df.pivot_table(index='date', columns='keyword', values='sentiment', aggfunc='mean')
df_p = df.pivot_table(index='date', columns='Category', values='sentiment', aggfunc='mean')

df_p.to_csv("temp/temp4.csv")


In [8]:
df= pd.read_csv("temp/temp4.csv")

df = df.reset_index()
df_c = df[df['date'].str.contains( "'" )==False ]

df_c['date'] = pd.to_datetime(df_c['date'], format='%Y-%m-%d %H:%M:%S')
df_c = df_c.set_index('date')
agg = df_c.resample('D').mean()

cat_list_roll = []
for cat in category_list:   
    agg[cat +"_roll"] = agg[cat].rolling(30).mean()
    cat_list_roll.append([cat +"_roll"])

print(cat_list_roll)
print(agg.head())

agg.to_csv("temp/wed_export.csv")
import plotly.express as px
fig = px.line(agg, x=agg.index, y=["general_roll","prices_roll","personal_roll","policy_roll"], title='PC Sentiment')
fig.show()

[['policy_roll'], ['personal_roll'], ['general_roll'], ['prices_roll']]
             index   general  personal    policy    prices  policy_roll  \
date                                                                      
2018-12-31   297.5  0.121821 -0.049068  0.433975 -0.012023          NaN   
2019-01-01   912.0  0.059233 -0.104917 -0.053041  0.029007          NaN   
2019-01-02  1560.0  0.031059 -0.025037  0.121668 -0.110409          NaN   
2019-01-03  2219.5  0.022706  0.026539  0.253673 -0.003059          NaN   
2019-01-04  2888.0 -0.146752  0.220486  0.160250  0.047942          NaN   

            personal_roll  general_roll  prices_roll  
date                                                  
2018-12-31            NaN           NaN          NaN  
2019-01-01            NaN           NaN          NaN  
2019-01-02            NaN           NaN          NaN  
2019-01-03            NaN           NaN          NaN  
2019-01-04            NaN           NaN          NaN  


In [ ]:
df_p = pd.read_csv("temp/temp4.csv")

df = df_p.reset_index()
df_c = df[df['date'].str.contains( "'" )==False ]

df_c['date'] = pd.to_datetime(df_c['date'], format='%Y-%m-%d %H:%M:%S')
df_c = df_c.set_index('date')
agg = df_c.resample('M').mean()

fig = px.line(agg, x=agg.index, y=["Better financially"], title='PC Sentiment')
fig.show()

"""
agg.to_csv("temp.csv")

agg = agg.reset_index()

start_date = '2017-01-01'
end_date = '2022-10-9'
mask = (agg['date'] > start_date) & (agg['date'] <= end_date)
agg = agg.loc[mask]

save_date = agg[['date']].copy()

agg = agg.drop(columns = ['date','nan'])
"""



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

agg_full = agg.apply(lambda row: row.fillna(row.mean()), axis=1)

#prepare the input series (x) by detrending and standardizing
x = StandardScaler().fit_transform(agg_full)

#Calculate the first  principal component & output explained variance
pca = PCA(n_components=1)
df_PC = pca.fit_transform(x)
print(pca.explained_variance_ratio_)


df_PC_merge = pd.DataFrame(df_PC, columns = ['Sentiment'])

df1 = pd.concat([save_date, df_PC_merge], axis=1)
df_final = df1.set_index('date')
df_final.to_csv("temp.csv")



[0.26063161]


In [ ]:
import plotly.express as px
df_final.to_csv("export.csv")
df_final['Sentiment'] = df_final['Sentiment']*(-1)
fig = px.line(df_final, x=df_final.index, y=["Sentiment"], title='PC Sentiment')
fig.show()

In [ ]:
#This is used for inspecting the values
start_date = '2017-12-01'
end_date = '2018-01-31'
mask = (df['date'] > start_date) & (df['date'] <= end_date)
df_trim = df.loc[mask]

df_trim = df_trim[df_trim['sentiment']>-0.8]
df_trim = df_trim[['date','tweet_clean','sentiment']].copy()
df_trim.to_csv("inspect_high.csv")